In [1]:
from scripts import dataset, tools, iterator
from models import alexnet, regressionnet
from tensorflow.contrib.data import Iterator
from IPython.display import clear_output
from copy import copy
from tqdm import tqdm
import math, os
import tensorflow as tf
import numpy as np
tools.etc.set_GPU("3")

In [2]:
net, loss_op, pose_loss_op, train_op = regressionnet.create_regression_net(data_shape=(227,227,3),optimizer_type='adadelta',num_joints=14)
    
with net.graph.as_default():
    saver = tf.train.Saver()

    train_it = dataset.met("/var/data/MET2/activity-met_n10_ub_train.csv", Fliplr=True, Shuffle=True)
    test_it = dataset.met("/var/data/MET2/activity-met_n10_ub_test.csv", Fliplr=True, Shuffle=True)

    summary_writer = tf.summary.FileWriter("./out/", net.sess.graph)
    summary_op = tf.summary.merge_all()

[Mirroring Images & Coordinates]: 100%|██████████| 53/53 [00:00<00:00, 745.33it/s]


In [3]:
def calculate_pcp(gt_joints, predicted_joints):
    predicted_joints = np.clip(predicted_joints.reshape(len(predicted_joints),14,2), -0.5, 0.5)
    pred_canonical = tools.pose.convert2canonical(predicted_joints)
    gt_canonical = tools.pose.convert2canonical(gt_joints)

    pcp_value=tools.pose.eval_strict_pcp(joints_gt=gt_canonical,predicted_joints=pred_canonical)
    average_pcp = tools.pose.average_pcp_left_right_limbs(pcp_value)
    return average_pcp

In [4]:
iter_num = 100000
snapshot_step = 5000
global_step = None
cur_train_op = train_op

In [5]:
with tf.device("/gpu:0"):
    with tqdm(total = iter_num) as pbar:
        for step in range(iter_num):
                
            tr_cost = 0.
            tr_cnt = 0
            tr_acc = np.zeros((7),np.float)
            for n in range(2):
                x=train_it.img_set[:476] if n==0 else train_it.img_set[476:]
                y=train_it.coor_set.reshape(952,-1)[:476] if n == 0 else train_it.coor_set.reshape(952,-1)[476:]
                v=train_it.joint_is_valid[:476] if n==0 else train_it.joint_is_valid[476:]
                global_step, summary_str, _, loss_value, score = net.sess.run(
                    [net.global_iter_counter,summary_op,cur_train_op,pose_loss_op, net.fc_regression],
                    feed_dict={net.x : x,
                               'PoseInput/joints_ground_truth:0' : y,
                               'PoseInput/joints_is_valid:0': v,
                               'lr/conv_lr:0': 1.,
                               'lr/fc_lr:0': 1.})
                tr_cost+=loss_value
                tr_cnt+=1
                tr_acc+=calculate_pcp(y[0].reshape(-1,14,2),score)[0]
                
            
            loss_value, score = net.sess.run(
                [pose_loss_op,net.fc_regression],
                feed_dict={net.x : test_it.img_set,
                           'PoseInput/joints_ground_truth:0' : test_it.coor_set.reshape(len(test_it.coor_set),-1),
                           'PoseInput/joints_is_valid:0': test_it.joint_is_valid,
                           'lr/conv_lr:0': 1.,
                           'lr/fc_lr:0': 1.})
            te_acc = calculate_pcp(test_it.coor_set,score)[0]
            
            if step%snapshot_step==0 and step !=0:
                saver.save(net.sess, "./out/alexnet_"+str(step)+".ckpt")
            
            clear_output()
            
            pbar.update(1)
            pbar.set_description("[ Step : "+str(step+1)+"]")
            pbar.set_postfix_str(" Train Loss : "+str(tr_cost/tr_cnt)+str(tr_acc)+" Test Loss : "+str(loss_value)+str(te_acc))

[ Step : 100000]: 100%|██████████| 100000/100000 [46:59:48<00:00,  1.69s/it,  Train Loss : 36.113243103027344[0. 0. 0. 0. 0. 0. 0.] Test Loss : 2681.6362[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
